# Viscous remanent magnetization and thermal remanent magnetization

**Reference:** Chapter 7: How Rocks Get and Stay Magnetized: https://pmagpy.github.io/Essentials-JupyterBook/chapter7/

In this class period, we will further explore the **Néel Theory** of single-domain grains. Recall that the stability of a magnetic memory depends on the competition between:
1.  **Thermal Energy ($kT$):** Tries to randomize the magnetic moment.
2.  **Anisotropy Energy ($Kv$):** Tries to lock the moment along an "easy" axis.

The relaxation time $\tau$ (how long a magnetization persists) is given by:

$$\tau = \tau_0 \exp \left( \frac{Kv}{k_BT} \right)$$

Where:
* $\tau_0$: The frequency factor (inverse of attempt frequency, $\approx 10^{-9}$ s).
* $K$: Anisotropy constant (J/m³).
* $v$: Grain volume (m³).
* $k_B$: Boltzmann constant ($1.38 \times 10^{-23}$ J/K).
* $T$: Temperature (K).

## Import our tools

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Ellipse, PathPatch
from matplotlib.path import Path

# --- Physical Constants ---
k_B = 1.380649e-23  # Boltzmann constant (J/K)
tau_0 = 1e-9        # Frequency factor (seconds), typical value for magnetite

# Wong (2011) Palette: Blue, Orange, Vermillion
colors = ['#0072B2', '#E69F00', '#D55E00']

### Solving for Critical Volume

We want to  to determine the specific grain volume ($v$) and anisotropy energy required to maintain magnetization for a given duration ($\tau$).

By rearranging the Néel equation $\tau = \tau_0 \exp \left( \frac{Kv}{kT} \right)$, we can solve for $v$:

$$\ln \left( \frac{\tau}{\tau_0} \right) = \frac{Kv}{kT} \quad \Rightarrow \quad v = \frac{kT}{K} \ln \left( \frac{\tau}{\tau_0} \right)$$

The function below implements this relationship, allowing us to calculate the critical volume for any combination of temperature, anisotropy, and desired stability time.

In [ ]:
def calculate_volume_for_relaxation(tau, K, T):
    """Rearranges the Neel equation to solve for Volume (v).

    Calculates the grain volume required to maintain magnetization for a given
    relaxation time, based on the anisotropy and temperature.

    Parameters:
        tau (float): The relaxation time in seconds.
        K (float or numpy.ndarray): The anisotropy constant in J/m^3.
        T (float): The temperature in Kelvin.

    Returns:
        float or numpy.ndarray: The calculated grain volume in m^3.
    """
    k_B = 1.380649e-23  # Boltzmann constant (J/K)
    v = (k_B * T * np.log(tau / tau_0)) / K
    return v

## Calculating relaxation time curves

With this function, we can calculate the stability curves for room temperature ($300$ K). We determine the critical volume ($v$) required for a grain to hold its magnetization for:
* **100 s:** Laboratory timescale (blocking boundary).
* **1 Myr:** Geological timescale.
* **4.5 Gyr:** The age of the Earth.

In [ ]:
T_room = 300  # Kelvin

# Define relaxation times of interest
taus = {
    '100 s': 100,
    '1 Myr': 1e6 * 365 * 24 * 3600,
    '4.5 Gyr': 4.5e9 * 365 * 24 * 3600
}

# Generate K values
K_axis_kJ = np.linspace(0.01, 6.0, 1000)
K_axis_J = K_axis_kJ * 1000

# Calculate volume curves
v_curves = {}
for label, t_val in taus.items():
    v_m3 = calculate_volume_for_relaxation(t_val, K_axis_J, T_room)
    v_curves[label] = v_m3 * 1e21

## Plot the "iso-$\tau$" curves

In [ ]:
def plot_neel_diagram(T=300, K_kJ=None, v_curves=None,
                      colors=None, figsize=(8, 5), xlim=(0, 4), ylim=(0, 0.5)):
    """Plot Néel diagram showing iso-tau curves at temperature T.

    Parameters
    ----------
    T : float
        Temperature in Kelvin.
    K_kJ : array-like
        Anisotropy energy density values in kJ/m³.
    v_curves : dict
        Pre-computed volume curves {label: v_values_in_zm3}.
    colors : list
        Colors for each tau curve (Wong palette default).
    figsize : tuple
        Figure size.
    xlim, ylim : tuple
        Axis limits.

    Returns
    -------
    fig, ax : matplotlib Figure and Axes
    """
    if colors is None:
        colors = ['#0072B2', '#E69F00', '#D55E00']

    fig, ax = plt.subplots(figsize=figsize)

    for (label, v_vals), color in zip(v_curves.items(), colors):
        ax.plot(K_kJ, v_vals,
                label=rf'$\tau = \mathrm{{{label.replace(" ", "~")}}}$',
                color=color, linewidth=2.5)

    ax.set_xlabel(r'Anisotropy energy density ($kJ/m^3$)', fontsize=12)
    ax.set_ylabel(r'Grain volume ($zm^3$)', fontsize=12)
    ax.set_title(f'Stability of Magnetite Grains at {T} K', fontsize=14)
    ax.set_ylim(ylim)
    ax.set_xlim(xlim)
    ax.legend(fontsize=11)
    ax.grid(True, linestyle='--', alpha=0.5)

    return fig, ax

In [ ]:
fig, ax = plot_neel_diagram(T=T_room, K_kJ=K_axis_kJ, v_curves=v_curves)
plt.tight_layout()
plt.show()

### Visualizing Stability Fields (VRM)

The plot below recreates the concepts from **Figure 7.3** (and 7.2b). We visualize lines of equal relaxation time ($\tau$) in $v-K$ space at room temperature ($300$ K) and then plot a hypothetical population of grains atop of them.

* **Superparamagnetic (SP):** Grains with low volume or anisotropy (bottom-left) have short $\tau$ (< 100 s). They flip rapidly and are unstable.
* **Stable Single Domain (SSD):** Grains with high volume or anisotropy (top-right) have long $\tau$ (> 100 s). They are blocked and stable over geological time.
* **The Population:** The yellow ellipses represent a statistical distribution of grains. Some of the grains are in the SP field and some in the SSD field.

In [ ]:
fig, ax = plot_neel_diagram(T=T_room, K_kJ=K_axis_kJ, v_curves=v_curves, figsize=(10, 7))

center_x, center_y = 1.3, 0.19
ellipse_params = [
    (1.6, 0.30, 0.15),
    (1.0, 0.20, 0.35),
    (0.5, 0.10, 0.65),
]
for width, height, alpha in ellipse_params:
    ellipse = patches.Ellipse((center_x, center_y), width, height,
                              angle=0, linewidth=1, edgecolor=None,
                              facecolor='gold', alpha=alpha)
    ax.add_patch(ellipse)

ax.text(0.6, 0.025, 'Superparamagnetic', rotation=-45, fontsize=12, ha='center', color='#333333')
ax.text(2.2, 0.25, 'Stable Single Domain', rotation=-45, fontsize=12, ha='center', color='#333333')
plt.show()

## SP grains

In [ ]:
fig, ax = plot_neel_diagram(T=T_room, K_kJ=K_axis_kJ, v_curves=v_curves, figsize=(10, 7))

center_x, center_y = 1.3, 0.19
width, height = 1.6, 0.30

sp_verts = list(zip(K_axis_kJ, v_curves['100 s'])) + [(K_axis_kJ[-1], 0), (K_axis_kJ[0], 0)]
ssd_verts = list(zip(K_axis_kJ, v_curves['100 s'])) + [(K_axis_kJ[-1], 0.5), (K_axis_kJ[0], 0.5)]

ellipse_sp = Ellipse((center_x, center_y), width, height, angle=0,
                     facecolor='lightblue', edgecolor='gray', linewidth=1,
                     alpha=0.5, hatch='///')
ax.add_patch(ellipse_sp)
ellipse_sp.set_clip_path(PathPatch(Path(sp_verts), transform=ax.transData))

ellipse_ssd = Ellipse((center_x, center_y), width, height, angle=0,
                      facecolor='gold', edgecolor='gray', linewidth=1, alpha=0.5)
ax.add_patch(ellipse_ssd)
ellipse_ssd.set_clip_path(PathPatch(Path(ssd_verts), transform=ax.transData))

ellipse_outline = Ellipse((center_x, center_y), width, height, angle=0,
                          facecolor='none', edgecolor='gray', linewidth=1.5)
ax.add_patch(ellipse_outline)

ax.text(0.6, 0.025, 'Superparamagnetic', rotation=-45, fontsize=12, ha='center', color='#333333')
ax.text(2.2, 0.25, 'Stable Single Domain', rotation=-45, fontsize=12, ha='center', color='#333333')
plt.show()

In [ ]:
# @title Interactive VRM Simulation { display-mode: "form" }

import plotly.graph_objects as go
import numpy as np

# --- 1. Physics & Constants ---
k_B = 1.380649e-23
tau_0 = 1e-9

# Grain population parameters
center_x, center_y = 1.3, 0.19
ellipse_width, ellipse_height = 1.6, 0.30

# Reference tau lines
taus_ref = {'100 s': 100, '1 Myr': 1e6*3.15e7, '4.5 Gyr': 4.5e9*3.15e7}
tau_colors = {'100 s': '#0072B2', '1 Myr': '#E69F00', '4.5 Gyr': '#D55E00'}

T_room = 300  # K

K_axis_kJ = np.linspace(0.01, 5.0, 300)
K_axis_J = K_axis_kJ * 1000

# --- 2. Helper Functions ---
def calculate_v_curve(tau, T, K_array_J):
    K_safe = np.maximum(K_array_J, 1e-5)
    v_m3 = (k_B * T * np.log(tau / tau_0)) / K_safe
    return v_m3 * 1e21

def get_ellipse_coords(cx, cy, w, h, n=200):
    theta = np.linspace(0, 2*np.pi, n)
    return cx + (w/2)*np.cos(theta), cy + (h/2)*np.sin(theta)

def clip_ellipse_below_curve(cx, cy, w, h, K_kJ, v_curve, n=500):
    theta = np.linspace(0, 2*np.pi, n)
    ex = cx + (w/2)*np.cos(theta)
    ey = cy + (h/2)*np.sin(theta)
    v_at_ex = np.interp(ex, K_kJ, v_curve, left=np.inf, right=np.inf)
    pts_x, pts_y = [], []
    for i in range(n):
        pts_x.append(ex[i])
        pts_y.append(min(ey[i], v_at_ex[i]))
    return np.array(pts_x), np.array(pts_y)

def clip_ellipse_above_curve(cx, cy, w, h, K_kJ, v_curve, n=500):
    theta = np.linspace(0, 2*np.pi, n)
    ex = cx + (w/2)*np.cos(theta)
    ey = cy + (h/2)*np.sin(theta)
    v_at_ex = np.interp(ex, K_kJ, v_curve, left=np.inf, right=np.inf)
    pts_x, pts_y = [], []
    for i in range(n):
        pts_x.append(ex[i])
        pts_y.append(max(ey[i], v_at_ex[i]))
    return np.array(pts_x), np.array(pts_y)

# --- 3. Clean time steps with round labels ---
time_steps = [
    (1, "1 s"), (3, "3 s"), (10, "10 s"), (30, "30 s"), (100, "100 s"),
    (300, "5 min"), (600, "10 min"), (1800, "30 min"), (3600, "1 hr"), (36000, "10 hr"),
    (86400, "1 day"), (604800, "1 week"), (2.628e6, "1 month"), (7.884e6, "3 months"),
    (3.15e7, "1 yr"), (1.58e8, "5 yr"), (3.15e8, "10 yr"), (1.58e9, "50 yr"),
    (3.15e9, "100 yr"), (1.58e10, "500 yr"), (3.15e10, "1 kyr"), (3.15e11, "10 kyr"),
    (1.58e12, "50 kyr"), (3.15e12, "100 kyr"), (1.58e13, "500 kyr"),
    (3.15e13, "1 Myr"), (3.15e14, "10 Myr"), (1.58e15, "50 Myr"),
    (3.15e15, "100 Myr"), (1.58e16, "500 Myr"), (3.15e16, "1 Gyr"),
]

# --- 4. Build Frames ---
frames = []

for i, (t_elapsed, t_label) in enumerate(time_steps):
    frame_data = []

    v_boundary = calculate_v_curve(t_elapsed, T_room, K_axis_J)

    # A. SSD portion (gold)
    ax_above, ay_above = clip_ellipse_above_curve(
        center_x, center_y, ellipse_width, ellipse_height, K_axis_kJ, v_boundary)
    frame_data.append(go.Scatter(
        x=ax_above, y=ay_above, fill='toself', mode='lines',
        line=dict(width=0.5, color='gray'),
        fillcolor='rgba(255, 215, 0, 0.5)',
        hoverinfo='skip', showlegend=False
    ))

    # B. VRM/SP portion (light blue)
    bx_below, by_below = clip_ellipse_below_curve(
        center_x, center_y, ellipse_width, ellipse_height, K_axis_kJ, v_boundary)
    frame_data.append(go.Scatter(
        x=bx_below, y=by_below, fill='toself', mode='lines',
        line=dict(width=0.5, color='gray'),
        fillcolor='rgba(173, 216, 230, 0.6)',
        hoverinfo='skip', showlegend=False
    ))

    # C. Ellipse outline
    ox, oy = get_ellipse_coords(center_x, center_y, ellipse_width, ellipse_height)
    frame_data.append(go.Scatter(
        x=ox, y=oy, mode='lines',
        line=dict(width=1.5, color='gray'),
        hoverinfo='skip', showlegend=False
    ))

    # D. Reference tau lines
    for label, t_val in taus_ref.items():
        v_vals = calculate_v_curve(t_val, T_room, K_axis_J)
        frame_data.append(go.Scatter(
            x=K_axis_kJ, y=v_vals,
            mode='lines', line=dict(color=tau_colors[label], width=2.5),
            name=f'τ = {label}'
        ))

    # E. Current tau=t boundary (black dashed, drawn LAST so on top)
    frame_data.append(go.Scatter(
        x=K_axis_kJ, y=np.clip(v_boundary, 0, 0.5),
        mode='lines', line=dict(color='black', width=2, dash='dash'),
        name='τ = t (blocking boundary)', showlegend=True
    ))

    frames.append(go.Frame(data=frame_data, name=str(i)))

# --- 5. Create Figure ---
fig = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        title="VRM Acquisition: Grain Population Remagnetization Over Time",
        xaxis=dict(title="Anisotropy Energy Density (kJ/m³)", range=[0, 4]),
        yaxis=dict(title="Grain Volume (zm³)", range=[0, 0.5]),
        template="simple_white",
        height=600,
        legend=dict(x=0.55, y=0.98, bgcolor='rgba(255,255,255,0.8)'),

        sliders=[{
            'active': 0,
            'currentvalue': {"prefix": "Elapsed Time: "},
            'pad': {"t": 50},
            'steps': [
                {
                    'method': 'animate',
                    'args': [[str(i)], {
                        'mode': 'immediate',
                        'frame': {'duration': 0, 'redraw': True},
                        'transition': {'duration': 0}
                    }],
                    'label': t_label
                } for i, (_, t_label) in enumerate(time_steps)
            ]
        }],

        annotations=[
            dict(x=0.4, y=0.05, text="VRM acquired<br>(Superparamagnetic)",
                 showarrow=False, font=dict(size=12, color="#0072B2")),
            dict(x=2.5, y=0.25, text="Stable remanence<br>(Original NRM)",
                 showarrow=False, font=dict(size=12, color="#B8860B"))
        ]
    ),
    frames=frames
)

fig.show()

## How TRM is Acquired

TRM is acquired when a rock cools from high temperature. This changes the stability diagram in two simultaneous ways:

1.  **The Curves Move:** As $T$ decreases, the thermal energy $kT$ decreases. Lower energy barriers are needed to block the moment, so the iso-$\tau$ curves shift.
2.  **The Grains Move:** The grain volume $v$ stays constant, but the anisotropy constant $K$ increases as the rock cools.
    * $K(T) \propto M_s(T)^2$ (for shape anisotropy).
    * As $T$ drops, $M_s$ rises, so **$K$ increases**. The grain population moves to the **right**.

In [ ]:
def plot_neel_curves(T_k, K_kJ, K_J, taus, tau_colors, ax,
                     linestyle='-', alpha=1.0, label_suffix='', show_legend_labels=True):
    """Plot iso-tau curves on a given axis for a specific temperature.

    Parameters
    ----------
    T_k : float
        Temperature in Kelvin.
    K_kJ, K_J : array-like
        Anisotropy values in kJ/m³ and J/m³.
    taus : dict
        {label: tau_seconds}.
    tau_colors : dict
        {label: color}.
    ax : matplotlib Axes
    linestyle : str
    alpha : float
    label_suffix : str
        Appended to legend labels (e.g. ' (550°C)').
    show_legend_labels : bool
        If False, sets label to '_nolegend_'.
    """
    for label, t_val in taus.items():
        v_m3 = calculate_volume_for_relaxation(t_val, K_J, T_k)
        leg = f'$\\tau$ = {label}{label_suffix}' if show_legend_labels else '_nolegend_'
        ax.plot(K_kJ, v_m3 * 1e21, color=tau_colors[label],
                linestyle=linestyle, linewidth=2.5, alpha=alpha, label=leg)


def format_neel_axes(ax, xlabel=True, ylabel=True, title='', xlim=(0, 4), ylim=(0, 0.5)):
    """Apply standard formatting to a Néel diagram axis."""
    if xlabel:
        ax.set_xlabel(r'Anisotropy energy density ($kJ/m^3$)', fontsize=12)
    if ylabel:
        ax.set_ylabel(r'Grain volume ($zm^3$)', fontsize=12)
    if title:
        ax.set_title(title, fontsize=14)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.grid(True, linestyle='--', alpha=0.3)


# === Common setup ===
taus = {'100 s': 100, '1 Myr': 1e6*3.15e7, '4.5 Gyr': 4.5e9*3.15e7}
tau_colors = {'100 s': '#0072B2', '1 Myr': '#E69F00', '4.5 Gyr': '#D55E00'}
T_high_K, T_low_K = 550 + 273.15, 20 + 273.15

# === Plot 1: Side-by-side ===
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
plot_neel_curves(T_high_K, K_axis_kJ, K_axis_J, taus, tau_colors, ax1)
format_neel_axes(ax1, title='High Temperature (550°C)')
ax1.legend()

plot_neel_curves(T_low_K, K_axis_kJ, K_axis_J, taus, tau_colors, ax2)
format_neel_axes(ax2, ylabel=False, title='Low Temperature (20°C)')
plt.tight_layout()
plt.show()

# === Plot 2: Overlay with labels ===
manual_settings = {
    '100 s (20°C)':    {'x': 1.1, 'y': 0.08, 'angle': -25},
    '1 Myr (20°C)':    {'x': 1.3, 'y': 0.18, 'angle': -42},
    '4.5 Gyr (20°C)':  {'x': 0.8, 'y': 0.30, 'angle': -58},
    '100 s (550°C)':   {'x': 3.3, 'y': 0.08, 'angle': -12},
    '1 Myr (550°C)':   {'x': 3.0, 'y': 0.18, 'angle': -25},
    '4.5 Gyr (550°C)': {'x': 2.3, 'y': 0.30, 'angle': -38},
}

fig, ax = plt.subplots(figsize=(12, 8))
for T_k, lstyle, alpha_val, suffix in [(T_high_K, '--', 0.6, ' (550°C)'),
                                        (T_low_K, '-', 1.0, ' (20°C)')]:
    plot_neel_curves(T_k, K_axis_kJ, K_axis_J, taus, tau_colors, ax,
                     linestyle=lstyle, alpha=alpha_val, show_legend_labels=False)
    for label in taus:
        full_label = f'{label}{suffix}'
        if full_label in manual_settings:
            cfg = manual_settings[full_label]
            ax.text(cfg['x'], cfg['y'], full_label,
                    color=tau_colors[label], fontsize=13, fontweight='bold',
                    rotation=cfg['angle'], ha='center', va='center', alpha=alpha_val,
                    bbox=dict(facecolor='white', alpha=1.0, edgecolor='none', pad=1.2))

format_neel_axes(ax, title='Thermal Stability Boundaries: 20°C vs 550°C', xlim=(0, 4.5))
plt.tight_layout()
plt.show()

### The Grains Move (Why Anisotropy Changes with Temperature)

Before simulating TRM, we need to understand **why** the grains become "harder" (higher anisotropy) as they cool.

1.  **Saturation Magnetization ($M_s$):** As temperature decreases, thermal fluctuations reduce, and atomic spins align more perfectly. $M_s$ increases.
2.  **Anisotropy Energy ($K$):** We assume the grains are elongated (**Shape Anisotropy**). The energy barrier comes from magnetostatic forces between the magnetic poles at the grain ends. Since the pole strength depends on $M_s$, the energy barrier scales with the *square* of the magnetization:
    $$K_{shape} \propto M_s(T)^2$$

This means that as a rock cools from the Curie temperature ($T_c$), the anisotropy ($K$) grows faster than the magnetization itself. This rapid increase in $K$ is places an important role in "blocking" the grains during cooling.

In [ ]:
# Physical Constants for Magnetite
Tc = 580.0           # Curie Temp (Celsius)
Ms_0 = 480000.0      # Saturation Magnetization at Room Temp (A/m)
K_0  = 25000.0       # Anisotropy Constant at Room Temp (J/m^3)
gamma = 0.38         # Exponent for Magnetite

T_celsius = np.linspace(0, Tc, 200)
T_kelvin = T_celsius + 273.15
Tc_kelvin = Tc + 273.15
T_room_k = 20 + 273.15 # Reference Room Temp (20 C)

# Calculate Ms(T) in A/m
# The formula: Ms(T) = Ms_0 * ((Tc - T) / (Tc - T_room))^gamma
ms_actual = Ms_0 * ((Tc_kelvin - T_kelvin) / (Tc_kelvin - T_room_k))**gamma
ms_actual = np.maximum(ms_actual, 0) # Physics doesn't allow negative Ms

# For shape anisotropy, K = 1/2 * dN * Ms^2.
# Therefore, K(T) scales with the square of Ms(T).
k_actual = K_0 * (ms_actual / Ms_0)**2

# Normalization (Relative to Room Temp 20 C)
ms_norm = ms_actual / Ms_0
k_norm = k_actual / K_0

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot Saturation Magnetization
ax1.plot(T_celsius, ms_norm, color='#0072B2', linewidth=3)
ax1.set(title=r'Saturation Magnetization ($M_s$)',
        ylabel=r'Normalized $M_s(T) / M_s(20^{\circ}C)$',
        xlabel='Temperature (°C)', xlim=(0, 600), ylim=(0, 1.1))
ax1.grid(True, linestyle='--', alpha=0.5)

# Plot Anisotropy Energy
ax2.plot(T_celsius, k_norm, color='#D55E00', linewidth=3)
ax2.set(title=r'Anisotropy Energy Density ($K$)',
        ylabel=r'Normalized $K(T) / K(20^{\circ}C)$',
        xlabel='Temperature (°C)', xlim=(0, 600), ylim=(0, 1.1))
ax2.grid(True, linestyle='--', alpha=0.5)

ax2.text(80, 0.5, r'$K \propto M_s^2$', fontsize=18, color='#D55E00')
ax2.text(80, 0.3, 'Energy barrier grows\nrapidly with cooling!', fontsize=12, color='#555')

plt.tight_layout()
plt.show()

## Putting it all together to simulate TRM

In [ ]:
# @title Interactive TRM Simulation { display-mode: "form" }

import plotly.graph_objects as go
import numpy as np

# --- 1. Physics & Constants ---
k_B = 1.380649e-23
tau_0 = 1e-9
Tc_magnetite = 853
gamma = 0.38

T_room_K = 293.15
ref_center_x = 1.3
ref_center_y = 0.19

taus = {'100 s': 100, '1 Myr': 1e6*3.15e7, '4.5 Gyr': 4.5e9*3.15e7}
tau_colors = {'100 s': '#0072B2', '1 Myr': '#E69F00', '4.5 Gyr': '#D55E00'}

ellipse_width_ref = 1.6
ellipse_height = 0.30

# --- 2. Helper Functions ---
def get_ms_ratio(T_kelvin):
    if T_kelvin >= Tc_magnetite: return 0
    ms_T = (1 - T_kelvin/Tc_magnetite)**gamma
    ms_room = (1 - T_room_K/Tc_magnetite)**gamma
    return ms_T / ms_room

def get_ellipse_coords(cx, cy, w, h, n=200):
    theta = np.linspace(0, 2*np.pi, n)
    return cx + (w/2)*np.cos(theta), cy + (h/2)*np.sin(theta)

def calculate_v_curve(tau, T_kelvin, K_array_J):
    K_safe = np.maximum(K_array_J, 1e-5)
    v_m3 = (k_B * T_kelvin * np.log(tau / tau_0)) / K_safe
    return v_m3 * 1e21

def clip_ellipse_above_curve(cx, cy, w, h, K_kJ, v_curve, n=500):
    """Ellipse portion ABOVE the curve (blocked/SSD grains)."""
    theta = np.linspace(0, 2*np.pi, n)
    ex = cx + (w/2)*np.cos(theta)
    ey = cy + (h/2)*np.sin(theta)
    v_at_ex = np.interp(ex, K_kJ, v_curve, left=np.inf, right=np.inf)
    px, py = [], []
    for i in range(n):
        px.append(ex[i])
        py.append(max(ey[i], v_at_ex[i]))
    return np.array(px), np.array(py)

def clip_ellipse_below_curve(cx, cy, w, h, K_kJ, v_curve, n=500):
    """Ellipse portion BELOW the curve (unblocked/SP grains)."""
    theta = np.linspace(0, 2*np.pi, n)
    ex = cx + (w/2)*np.cos(theta)
    ey = cy + (h/2)*np.sin(theta)
    v_at_ex = np.interp(ex, K_kJ, v_curve, left=np.inf, right=np.inf)
    px, py = [], []
    for i in range(n):
        px.append(ex[i])
        py.append(min(ey[i], v_at_ex[i]))
    return np.array(px), np.array(py)

# --- 3. Build Frames ---
temps_c = np.arange(20, 580, 10)
temps_k = temps_c + 273.15

K_axis_kJ = np.linspace(0.01, 5.0, 300)
K_axis_J = K_axis_kJ * 1000

frames = []

for t_c, t_k in zip(temps_c, temps_k):
    frame_data = []

    ms_ratio = get_ms_ratio(t_k)
    shift_factor = ms_ratio**2
    current_cx = ref_center_x * shift_factor
    current_width = ellipse_width_ref * shift_factor

    # tau=100s boundary at this temperature
    v_boundary = calculate_v_curve(100, t_k, K_axis_J)

    # A. Blocked/SSD portion (gold) — above tau=100s
    ax_above, ay_above = clip_ellipse_above_curve(
        current_cx, ref_center_y, current_width, ellipse_height, K_axis_kJ, v_boundary)
    frame_data.append(go.Scatter(
        x=ax_above, y=ay_above, fill='toself', mode='lines',
        line=dict(width=0.5, color='gray'),
        fillcolor='rgba(255, 215, 0, 0.5)',
        hoverinfo='skip', showlegend=False
    ))

    # B. Unblocked/SP portion (light blue) — below tau=100s
    bx_below, by_below = clip_ellipse_below_curve(
        current_cx, ref_center_y, current_width, ellipse_height, K_axis_kJ, v_boundary)
    frame_data.append(go.Scatter(
        x=bx_below, y=by_below, fill='toself', mode='lines',
        line=dict(width=0.5, color='gray'),
        fillcolor='rgba(173, 216, 230, 0.6)',
        hoverinfo='skip', showlegend=False
    ))

    # C. Ellipse outline
    ox, oy = get_ellipse_coords(current_cx, ref_center_y, current_width, ellipse_height)
    frame_data.append(go.Scatter(
        x=ox, y=oy, mode='lines',
        line=dict(width=1.5, color='gray'),
        hoverinfo='skip', showlegend=False
    ))

    # D. Iso-tau curves (on top)
    for label, t_val in taus.items():
        v_vals = calculate_v_curve(t_val, t_k, K_axis_J)
        frame_data.append(go.Scatter(
            x=K_axis_kJ, y=v_vals,
            mode='lines', line=dict(color=tau_colors[label], width=3),
            name=f'τ = {label}'
        ))

    frames.append(go.Frame(data=frame_data, name=str(t_c)))

# --- 4. Create Figure (start at high T) ---
fig = go.Figure(
    data=frames[-1].data,
    layout=go.Layout(
        title="TRM Acquisition: Cooling from Curie Temperature",
        xaxis=dict(title="Anisotropy Energy Density (kJ/m³)", range=[0, 4.5]),
        yaxis=dict(title="Grain Volume (zm³)", range=[0, 0.5]),
        template="simple_white",
        height=600,
        legend=dict(x=0.55, y=0.98, bgcolor='rgba(255,255,255,0.8)'),

        sliders=[{
            'active': len(temps_c) - 1,
            'currentvalue': {"prefix": "Temperature: ", "suffix": " °C"},
            'pad': {"t": 50},
            'steps': [
                {
                    'method': 'animate',
                    'args': [[str(t)], {
                        'mode': 'immediate',
                        'frame': {'duration': 0, 'redraw': True},
                        'transition': {'duration': 0}
                    }],
                    'label': str(t)
                } for t in temps_c
            ]
        }],

        annotations=[
            dict(x=0.4, y=0.05, text="Unblocked (SP)",
                 showarrow=False, font=dict(size=12, color="#0072B2")),
            dict(x=3.5, y=0.4, text="Blocked (TRM acquired)",
                 showarrow=False, font=dict(size=12, color="#B8860B"))
        ]
    ),
    frames=frames
)

fig.show()